In [12]:
# Some exploratory work
import pandas as pd
import numpy as np
from sklearn import tree

In [2]:
# One way to read the data, but it's not the way we'll use for now

# Adapted from here:
#https://stackoverflow.com/questions/55129640/read-csv-into-a-dataframe-with-varying-row-lengths-using-pandas
df = pd.read_csv("processed_acl/kitchen/negative.review", header=None)
df = df[0].str.split(' ', expand=True)

In [3]:
master_dict = {}

def add_to_master_list(filename):
    num_lines = 0
    with open(filename) as file:
        for line in file:
            num_lines += 1
            words = line.rstrip().split(' ')
            for i in range(len(words)-1):
                word = words[i].split(":")[0]
                if word not in master_dict:
                    master_dict[word] = 1
    return num_lines

number_examples = {}
number_examples["negative"] = add_to_master_list("processed_acl/kitchen/negative.review")
number_examples["positive"] = add_to_master_list("processed_acl/kitchen/positive.review")

In [27]:
# A new version of filling the dictionary, which I hoped will have better runtime
# but does not appear to.
# Don't run this block for now.

# A list of the words and in the dictionary, keep track of the index for constant time access
master_list = list(master_dict.keys())
for i in range(len(master_list)):
    master_dict[master_list[i]] = i

# The training dataframe
num_training_examples = number_examples["negative"]+number_examples["positive"]
df_train = pd.DataFrame(columns = np.arange(len(master_list)))
df_rows = []

def add_words(filename, start_point):
    global df_train
    with open(filename) as file:
        new_row = np.zeros(len(master_list))
        for line in file:
            words = line.rstrip().split(' ')
            for i in range(len(words)-1):
                word = words[i].split(":")[0]
                count = int(words[i].split(":")[1])
                new_row[master_dict[word]] = count
            #df_train = pd.concat([df_train, pd.DataFrame(new_row).T])
            df_rows.append(pd.DataFrame(new_row).T)
add_words("processed_acl/kitchen/negative.review",0)
add_words("processed_acl/kitchen/positive.review",number_examples["negative"])

df_train = pd.concat(df_rows)

In [7]:
# Older version of the above block.
# Seems to be a little faster. I want a better way still to do this.

# A list of the words and in the dictionary, keep track of the index for constant time access
master_list = list(master_dict.keys())
for i in range(len(master_list)):
    master_dict[master_list[i]] = i

# The training dataframe
num_training_examples = number_examples["negative"]+number_examples["positive"]
df_train = pd.DataFrame(0, index=np.arange(num_training_examples), columns = np.arange(len(master_list)))
rating = []

def add_words(filename, start_point):
    with open(filename) as file:
        row_count = start_point
        for line in file:
            row_in_df = df_train.iloc[row_count]
            words = line.rstrip().split(' ')
            for i in range(len(words)-1):
                word = words[i].split(":")[0]
                count = int(words[i].split(":")[1])
                row_in_df[master_dict[word]] = count
            rating.append(words[-1].split("#")[2][1:])
            row_count += 1
            
add_words("processed_acl/kitchen/negative.review",0)
add_words("processed_acl/kitchen/positive.review",number_examples["negative"])

In [16]:
# Train the classifier
clf = tree.DecisionTreeClassifier()
clf = clf.fit(df_train, rating)

In [55]:
# Apply the classifier to the test data
test_results = [] # Predicted values
test_values = [] # Actual values

# Only do some of the lines for now
num_lines = 0
with open("processed_acl/kitchen/unlabeled.review") as file:
    for line in file:
        num_lines += 1
        if (num_lines < 10000):
            if (num_lines % 10 == 0):
                print(num_lines)
            word_vector = np.zeros(len(master_list))
            words = line.rstrip().split(' ')
            for i in range(len(words)-1):
                word = words[i].split(":")[0]
                if word in master_dict:
                    word_vector[master_dict[word]] = words[i].split(":")[1]
            test_values.append(words[-1].split("#")[2][1:])
            predicted_value = clf.predict(pd.DataFrame(word_vector).T)[0]
            test_results.append(predicted_value)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [59]:
# Calculate for
den = len(test_values)
num = len([0 for i in range(den) if test_values[i]==test_results[i]])
[num, den] # For the kitchen data set, it gets 4443/5945 right, or just under 75%.
num/den

0.7473507148864592

[4443, 5945]

['positive']